### Import the required libraries and modules:

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, Conv2DTranspose, Concatenate, Input
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import numpy as np
import cv2
from glob import glob
from sklearn.utils import shuffle
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, ReduceLROnPlateau, EarlyStopping, TensorBoard
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Recall, Precision
import numpy as np

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive", force_remount=True)
%cd /content/gdrive/MyDrive/medical_image_processing/

Mounted at /content/gdrive
/content/gdrive/MyDrive/medical_image_processing


In [ ]:
from fundal_image.utils import create_dir, get_filenames_sorted
from patches import generate_patches_images_only, reconstruct_patches

In [ ]:
loss_str = "bce"
with_augmentation = True
stride =  512

In [ ]:
aug_string = ""
if with_augmentation:
    aug_string = "_hvr"
root_path = f"/content/datasets/hrf_512_{stride}_{loss_str}{aug_string}/"
create_dir(f"{root_path}/images")
create_dir(f"{root_path}/groundtruth")

In [ ]:
images_path = f"{root_path}/images/"
gt_path = f"{root_path}/groundtruth"

In [ ]:
!cp /content/gdrive/MyDrive/medical_image_processing/fundal_image/datasets/HRF/images/*.* {root_path}/images
!cp /content/gdrive/MyDrive/medical_image_processing/fundal_image/datasets/HRF/groundtruth/*.* {root_path}/groundtruth/



In [ ]:
!ls {images_path}

01_dr.JPG  03_g.jpg   05_h.jpg	 08_dr.JPG  10_g.jpg   12_h.jpg   15_dr.JPG
01_g.jpg   03_h.jpg   06_dr.JPG  08_g.jpg   10_h.jpg   13_dr.JPG  15_g.jpg
01_h.jpg   04_dr.JPG  06_g.jpg	 08_h.jpg   11_dr.JPG  13_g.jpg   15_h.jpg
02_dr.JPG  04_g.jpg   06_h.jpg	 09_dr.JPG  11_g.jpg   13_h.jpg
02_g.jpg   04_h.jpg   07_dr.JPG  09_g.jpg   11_h.jpg   14_dr.JPG
02_h.jpg   05_dr.JPG  07_g.jpg	 09_h.jpg   12_dr.JPG  14_g.jpg
03_dr.JPG  05_g.jpg   07_h.jpg	 10_dr.JPG  12_g.jpg   14_h.jpg


In [ ]:

images_files, gt_files = get_filenames_sorted(images_path, gt_path)
print(len(images_files), len(gt_files))


45 45


In [ ]:
label = [_xtrain.rsplit("/", 1)[1].rsplit(".", 1)[0].rsplit("_", 1)[1] for _xtrain in images_files] 
print(label)

['dr', 'g', 'h', 'dr', 'g', 'h', 'dr', 'g', 'h', 'dr', 'g', 'h', 'dr', 'g', 'h', 'dr', 'g', 'h', 'dr', 'g', 'h', 'dr', 'g', 'h', 'dr', 'g', 'h', 'dr', 'g', 'h', 'dr', 'g', 'h', 'dr', 'g', 'h', 'dr', 'g', 'h', 'dr', 'g', 'h', 'dr', 'g', 'h']


In [ ]:
# train_test_split
xtrain = []
xtest = []
test_x = None
if test_x is None or len(test_x) == 0:
    print("Creating train test split")
    from sklearn.model_selection import train_test_split
    label = [_xtrain.rsplit("/", 1)[1].rsplit(".", 1)[0].rsplit("_", 1)[1] for _xtrain in images_files] 
    xtrain, xtest, ytrain, ytest = train_test_split(images_files, gt_files, stratify=label,  test_size=0.4, random_state=42)
else:
    # copy if there are any images without data_augmentation
    xtrain = [f for f in images_files if f.rsplit("/", 1)[1].rsplit(".", 1)[0] not in test_x]
    xtest = [f for f in images_files if f.rsplit("/", 1)[1].rsplit(".", 1)[0]  in test_x]

    # copy augmented images
    for f in images_files:
        name = f.rsplit("/", 1)[1].rsplit(".", 1)[0]
        if name.rsplit("_", 1)[1].isnumeric():
            if name.rsplit("_", 1)[0] in test_x:
                xtest.append(f)
            else:
                xtrain.append(f)




Creating train test split


In [ ]:
print(len(xtrain), len(xtest))


27 18


In [ ]:
from albumentations import HorizontalFlip, VerticalFlip, Rotate
def augment_data(images, masks, save_path, augment=True):
    H = 2336
    W = 3504

    for idx, (x, y) in tqdm(enumerate(zip(images, masks)), total=len(images)):
        """ Extracting names """
        name = x.rsplit("/", 1)[1].rsplit(".", 1)[0]
        print(name)
        """ Reading image and mask """
        x = cv2.imread(x, cv2.IMREAD_COLOR)
        y = cv2.imread(y, cv2.IMREAD_GRAYSCALE)

        if augment == True:
            aug = HorizontalFlip(p=1.0)
            augmented = aug(image=x, mask=y)
            x1 = augmented["image"]
            y1 = augmented["mask"]

            aug = VerticalFlip(p=1.0)
            augmented = aug(image=x, mask=y)
            x2 = augmented["image"]
            y2 = augmented["mask"]

            aug = Rotate(limit=45, p=1.0)
            augmented = aug(image=x, mask=y)
            x3 = augmented["image"]
            y3 = augmented["mask"]

            X = [x, x1, x2, x3]
            Y = [y, y1, y2, y3]

            X = [x, x1, x2, x3]
            Y = [y, y1, y2, y3]

        else:
            X = [x]
            Y = [y]

        index = 0
        for i, m in zip(X, Y):
            i = cv2.resize(i, (W, H))
            m = cv2.resize(m, (W, H))

            if len(X) == 1:
                tmp_image_name = f"{name}.png"
                tmp_mask_name = f"{name}.png"
            else:
                tmp_image_name = f"{name}_{index}.png"
                tmp_mask_name = f"{name}_{index}.png"

            image_path = os.path.join(save_path, "image", tmp_image_name)
            mask_path = os.path.join(save_path, "mask", tmp_mask_name)

            cv2.imwrite(image_path, i)
            cv2.imwrite(mask_path, m)

            index += 1

In [ ]:
create_dir(f"{root_path}/augmented/train/image/")
create_dir(f"{root_path}/augmented/train/mask/")

In [ ]:
from tqdm import tqdm

In [ ]:
augment_data(xtrain, ytrain, f"{root_path}/augmented/train/", True )

  0%|          | 0/27 [00:00<?, ?it/s]

13_g


  4%|▎         | 1/27 [00:01<00:48,  1.85s/it]

11_g


  7%|▋         | 2/27 [00:03<00:45,  1.81s/it]

...

 93%|█████████▎| 25/27 [00:46<00:03,  1.87s/it]

05_h


 96%|█████████▋| 26/27 [00:48<00:01,  1.86s/it]

03_dr


100%|██████████| 27/27 [00:50<00:00,  1.85s/it]


In [ ]:
xtrain, ytrain = get_filenames_sorted(f"{root_path}/augmented/train/image", f"{root_path}/augmented/train/mask")

In [ ]:
print(len(xtrain), len(ytrain))

108 108


In [ ]:
image_sizes = {"HRF": (2336, 3504), "drive": (584, 565), "chase_db": (960, 999)}
patch_size = {"HRF": (512, 512)}

dataset_name = "HRF"
num_pixels_to_add = [0, 0]
if (image_sizes["HRF"][0] % patch_size["HRF"][0]) > 0:
    num_pixels_to_add[0] = ((image_sizes["HRF"][0] // patch_size["HRF"][0]) + 1) * patch_size["HRF"][0]
    num_pixels_to_add[1] = ((image_sizes["HRF"][1] // patch_size["HRF"][1]) + 1) * patch_size["HRF"][1]

split_name = "split_2"
data_path = root_path
splits =["train","test"]


split = "train"
#for split in splits:
image_patches_path = f"{data_path}/{split_name}/{split}/image/"
create_dir(image_patches_path)
gt_patches_path = f"{data_path}/{split_name}/{split}/mask/"
create_dir(gt_patches_path)
im_patches_dict_train, metadata_train_df, gt_patches_dict_train = generate_patches_images_only(xtrain,
                                                                image_patches_path,
                                                                patch_size=patch_size["HRF"],
                                                                stride=stride,
                                                                zero_padding_size=num_pixels_to_add,
                                                                gt_files=ytrain,
                                                                save_path_gt=gt_patches_path)
metadata_train_df.to_csv(f"{data_path}/{split_name}/image_patches_512_{stride}_{split}.csv", index=False)



Generating patches for 108 images


100%|██████████| 108/108 [01:22<00:00,  1.32it/s]


In [ ]:
split = "test"
#for split in splits:
image_patches_path = f"{data_path}/{split_name}/{split}/image/"
create_dir(image_patches_path)
gt_patches_path = f"{data_path}/{split_name}/{split}/mask/"
create_dir(gt_patches_path)
im_patches_dict_test, metadata_test_df, gt_patches_dict_test = generate_patches_images_only(xtest,
                                                                image_patches_path,
                                                                patch_size=patch_size["HRF"],
                                                                stride=stride,
                                                                zero_padding_size=num_pixels_to_add,
                                                                gt_files=ytest,
                                                                save_path_gt=gt_patches_path)
metadata_test_df.to_csv(f"{data_path}/{split_name}/image_patches_512_{stride}_{split}.csv", index=False)


Generating patches for 18 images


100%|██████████| 18/18 [00:11<00:00,  1.52it/s]


In [ ]:
metadata_test_df.shape

(630, 7)

In [ ]:
metadata_train_df.head()

,image_filename,row,col,patch_height,patch_width,stride,num_fg_pixels
0,/content/datasets/hrf_512_512_bce_hvr//augment...,0,0,512,512,512,118326
1,/content/datasets/hrf_512_512_bce_hvr//augment...,0,512,512,512,512,200890
2,/content/datasets/hrf_512_512_bce_hvr//augment...,0,1024,512,512,512,204800
3,/content/datasets/hrf_512_512_bce_hvr//augment...,0,1536,512,512,512,204800
4,/content/datasets/hrf_512_512_bce_hvr//augment...,0,2048,512,512,512,204800


In [ ]:
import glob
split = "test"
image_patches_path = f"{data_path}/{split_name}/{split}/image/"

reconstructed_original_path = f"{data_path}/{split_name}/reconstructed/{split}/image"
create_dir(reconstructed_original_path)
reconstruct_patches(im_patches_dict_test,
                    metadata_test_df,
                    reconstructed_original_path
                    )


In [ ]:
reconstructed_gt_path = f"{data_path}/{split_name}/reconstructed/{split}/mask"

create_dir(reconstructed_gt_path)
reconstruct_patches(gt_patches_dict_test,
                    metadata_test_df,
                    reconstructed_gt_path,
                    )


In [ ]:
print([k for k in im_patches_dict_test.keys()])

['11_dr', '05_dr', '07_dr', '01_h', '15_g', '06_h', '15_h', '04_dr', '10_h', '06_g', '02_g', '05_g', '13_dr', '09_h', '03_g', '08_dr', '08_g', '12_h']


### Construct the Model:

In [ ]:
def conv_block(inputs, num_filters):
    x = Conv2D(num_filters, 3, padding="same")(inputs)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    x = Conv2D(num_filters, 3, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    return x

def encoder_block(inputs, num_filters):
    x = conv_block(inputs, num_filters)
    p = MaxPool2D((2, 2))(x)
    return x, p

def decoder_block(inputs, skip_features, num_filters):
    x = Conv2DTranspose(num_filters, (2, 2), strides=2, padding="same")(inputs)
    x = Concatenate()([x, skip_features])
    x = conv_block(x, num_filters)
    return x

def build_unet(input_shape):
    inputs = Input(input_shape)

    s1, p1 = encoder_block(inputs, 64)
    s2, p2 = encoder_block(p1, 128)
    s3, p3 = encoder_block(p2, 256)
    s4, p4 = encoder_block(p3, 512)

    b1 = conv_block(p4, 1024)

    d1 = decoder_block(b1, s4, 512)
    d2 = decoder_block(d1, s3, 256)
    d3 = decoder_block(d2, s2, 128)
    d4 = decoder_block(d3, s1, 64)

    outputs = Conv2D(1, 1, padding="same", activation="sigmoid")(d4)

    model = Model(inputs, outputs, name="UNET")
    return model

input_shape = (512, 512, 3)
model = build_unet(input_shape)
model.summary()

Model: "UNET"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 512, 512, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 512, 512, 64  1792        ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 512, 512, 64  256        ['conv2d[0][0]']                 
 alization)                     )                                                              

### Adding the Metrics:

In [ ]:
def iou(y_true, y_pred):
    def f(y_true, y_pred):
        intersection = (y_true * y_pred).sum()
        union = y_true.sum() + y_pred.sum() - intersection
        x = (intersection + 1e-15) / (union + 1e-15)
        x = x.astype(np.float32)
        return x
    return tf.numpy_function(f, [y_true, y_pred], tf.float32)

smooth = 1e-15
def dice_coef(y_true, y_pred):
    y_true = tf.keras.layers.Flatten()(y_true)
    y_pred = tf.keras.layers.Flatten()(y_pred)
    intersection = tf.reduce_sum(y_true * y_pred)
    return (2. * intersection + smooth) / (tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) + smooth)

def dice_loss(y_true, y_pred):
    return 1.0 - dice_coef(y_true, y_pred)

def DiceBCELoss(y_true, y_pred):
    
    inputs = tf.keras.layers.Flatten()(y_true)
    targets = tf.keras.layers.Flatten()(y_pred)

    intersection = tf.reduce_sum(inputs * targets)
    dice_loss = 1 - (2.*intersection + smooth)/(tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) + smooth)
    BCE = tf.keras.losses.BinaryCrossentropy()
    Dice_BCE = BCE(inputs, targets) + dice_loss

    return Dice_BCE

def bce(y_true, y_pred):
    
    inputs = tf.keras.layers.Flatten()(y_true)
    targets = tf.keras.layers.Flatten()(y_pred)
    BCE = tf.keras.losses.BinaryCrossentropy()
    Dice_BCE = BCE(inputs, targets)

    return Dice_BCE



### Training the Model:

In [ ]:
if loss_str == "bce":
    loss = bce
elif loss_str == "dice_bce":
    loss = DiceBCELoss
elif loss_str == "dice_loss":
    loss = dice_loss
else:
    raise Exception("Invalid value for loss_str")
    


In [ ]:
loss

<function __main__.bce>

In [ ]:
H = 512
W = 512

def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

def load_data(path):
    x = sorted(glob(os.path.join(path, "image", "*.png")))
    y = sorted(glob(os.path.join(path, "mask", "*.png")))
    return x, y

def shuffling(x, y):
    x, y = shuffle(x, y, random_state=42)
    return x, y

def read_image(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_COLOR)
    # x = cv2.resize(x, (W, H))
    x = x/255.0
    x = x.astype(np.float32)
    return x

def read_mask(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)  ## (512, 512)
    # x = cv2.resize(x, (W, H))
    x = x/255.0
    x = x.astype(np.float32)
    x = np.expand_dims(x, axis=-1)              ## (512, 512, 1)
    return x

def tf_parse(x, y):
    def _parse(x, y):
        x = read_image(x)
        y = read_mask(y)
        return x, y

    x, y = tf.numpy_function(_parse, [x, y], [tf.float32, tf.float32])
    x.set_shape([H, W, 3])
    y.set_shape([H, W, 1])
    return x, y

def tf_dataset(X, Y, batch_size=2):
    dataset = tf.data.Dataset.from_tensor_slices((X, Y))
    dataset = dataset.map(tf_parse)
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(4)
    return dataset

In [ ]:
from keras.metrics import AUC

In [ ]:
!ls

build		 fundal_image.egg-info	metrics.py   setup.py
classifier_data  hrf_bv_results		patches.py   tests.py
dist		 hrf_bv_results_sept_8	__pycache__  trained_models
fundal_image	 logs			README.md    trained_models_sept_8


In [ ]:
# root_path_gdrive = f"/content/gdrive/MyDrive/Blood_Vessel_Results/hrf_patches/hrf_512_512_{str(loss).split()[1]}/"
# print(root_path_gdrive)

In [ ]:
# model_dir = f"{root_path_gdrive}/trained_models/"
model_dir = f"{root_path}/trained_models/"

In [ ]:
create_dir(model_dir)

In [ ]:
%cd /content/datasets

/content/datasets


In [ ]:
model_dir

'/content/datasets/hrf_512_512_bce_hvr//trained_models/'

In [ ]:
num_epochs_completed = 0

In [ ]:
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import CustomObjectScope


In [ ]:
from glob import glob

In [ ]:
""" Seeding """
np.random.seed(42)
tf.random.set_seed(42)

""" Directory to save files """
create_dir(model_dir)

""" Hyperparameters """
batch_size = 8
lr = 1e-4
num_epochs = 50

last_model_path = os.path.join(f"{model_dir}", f"model_{num_epochs_completed}.h5")

model_path = os.path.join(f"{model_dir}", "model.h5")

csv_path = os.path.join(f"{model_dir}", "data.csv")

""" Dataset """
dataset_path = f"{data_path}/{split_name}"
train_path = os.path.join(dataset_path, "train")
valid_path = os.path.join(dataset_path, "test")

train_x, train_y = load_data(train_path)
train_x, train_y = shuffling(train_x, train_y)
valid_x, valid_y = load_data(valid_path)

print(f"Train: {len(train_x)} - {len(train_y)}")
print(f"Valid: {len(valid_x)} - {len(valid_y)}")

train_dataset = tf_dataset(train_x, train_y, batch_size=batch_size)
valid_dataset = tf_dataset(valid_x, valid_y, batch_size=batch_size)

train_steps = len(train_x)//batch_size
valid_setps = len(valid_x)//batch_size

if len(train_x) % batch_size != 0:
    train_steps += 1
if len(valid_x) % batch_size != 0:
    valid_setps += 1

""" Model """
model = build_unet((H, W, 3))
# with CustomObjectScope({"DiceBCELoss":DiceBCELoss, "dice_coef":dice_coef, "iou":iou}):
#     model = load_model(last_model_path)
model.compile(loss=loss, optimizer=Adam(lr), metrics=[dice_coef, iou, Recall(), Precision(), AUC()])

callbacks = [
    ModelCheckpoint(model_path, verbose=1, save_best_only=True, monitor="val_dice_coef", mode="max"),
    ReduceLROnPlateau(monitor="val_loss", factor=0.1, patience=5, min_lr=1e-6, verbose=1),
    CSVLogger(csv_path),
    TensorBoard(),
    EarlyStopping(monitor="val_dice_coef", patience=10, restore_best_weights=False, mode="max")
]

model.fit(
    train_dataset,
    epochs=num_epochs,
    validation_data=valid_dataset,
    steps_per_epoch=train_steps,
    validation_steps=valid_setps,
    callbacks=callbacks
)

Train: 3780 - 3780
Valid: 630 - 630
Epoch 1/50
473/473 [==============================] - ETA: 0s - loss: 0.1702 - dice_coef: 0.4220 - iou: 0.2752 - recall: 0.6320 - precision: 0.7121 - auc: 0.9371
Epoch 00001: val_dice_coef improved from -inf to 0.35512, saving model to /content/datasets/hrf_512_512_bce_hvr//trained_models/model.h5
473/473 [==============================] - 470s 955ms/step - loss: 0.1702 - dice_coef: 0.4220 - iou: 0.2752 - recall: 0.6320 - precision: 0.7121 - auc: 0.9371 - val_loss: 0.1493 - val_dice_coef: 0.3551 - val_iou: 0.2186 - val_recall: 0.4013 - val_precision: 0.8561 - val_auc: 0.9259 - lr: 1.0000e-04
Epoch 2/50
473/473 [==============================] - ETA: 0s - loss: 0.0941 - dice_coef: 0.5945 - iou: 0.4247 - recall: 0.7539 - precision: 0.8182 - auc: 0.9786
Epoch 00002: val_dice_coef improved from 0.35512 to 0.63239, saving model to /content/datasets/hrf_512_512_bce_hvr//trained_models/model.h5
473/473 [==============================] - 446s 944ms/step - lo

In [ ]:
num_epochs

50

### Evaluating the Model:

In [ ]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import numpy as np
import pandas as pd
import cv2
from glob import glob
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras.utils import CustomObjectScope
from sklearn.metrics import accuracy_score, f1_score, jaccard_score, precision_score, recall_score
from tensorflow.keras.models import load_model

In [ ]:
H = 512
W = 512

def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

def read_image(path):
    x = cv2.imread(path, cv2.IMREAD_COLOR)
    # x = cv2.resize(x, (W, H))
    ori_x = x
    x = x/255.0
    x = x.astype(np.float32)
    return ori_x, x

def read_mask(path):
    x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)  ## (512, 512)
    # x = cv2.resize(x, (W, H))
    ori_x = x
    x = x/255.0
    x = x.astype(np.int32)
    return ori_x, x

def load_data(path):
    x = sorted(glob(os.path.join(path, "image", "*.png")))
    y = sorted(glob(os.path.join(path, "mask", "*.png")))
    return x, y

def save_results(ori_x, ori_y, y_pred, save_image_path):
    line = np.ones((H, 10, 3)) * 255

    ori_y = np.expand_dims(ori_y, axis=-1)
    ori_y = np.concatenate([ori_y, ori_y, ori_y], axis=-1)

    y_pred = np.expand_dims(y_pred, axis=-1)
    y_pred = np.concatenate([y_pred, y_pred, y_pred], axis=-1) * 255

    cat_images = np.concatenate([ori_x, line, ori_y, line, y_pred], axis=1)
    cv2.imwrite(save_image_path, cat_images)

In [ ]:
import glob
supported_extensions = ["png", "jpg", "tif", "jpeg", "gif", "bmp"]

def image_files(path, file_prefix=None):
    all_image_files = []
    # print("path", path)
    # print("file_prefix", file_prefix)
    if file_prefix is None:
        file_names = list(glob.glob(path + "/*.*" ))
    else:
        prefix___ = path + "/" + file_prefix + ".*"
        #print("prefix", prefix___)
        file_names = list(glob.glob(prefix___))

    # print("filenames", file_names)

    for filename in file_names:
        ext = filename.rsplit(".", 1)[1]
        if ext.lower() in supported_extensions:
            all_image_files.append(filename)
    # print("All Image files", all_image_files)
    return sorted(all_image_files)


In [ ]:
from glob import glob
def get_filenames_sorted(image_path, gt_path=None):
    image_file_names = image_files(image_path)
    gt_file_names = None
    if gt_path is not None:
        # Get the test files also
        gt_file_names = sorted(image_files(gt_path))
    return image_file_names, gt_file_names

In [ ]:
results_path = f"{root_path}/results/"
prediction_path = f"{root_path}/predictions/"
prediction_np_path = f"{root_path}/predictions_np/"
create_dir(prediction_path)
create_dir(prediction_np_path)
create_dir(results_path)

In [ ]:
results_path

'/content/datasets/hrf_512_512_bce_hvr//results/'

In [ ]:
prediction_path

'/content/datasets/hrf_512_512_bce_hvr//predictions/'

In [ ]:
from copy import deepcopy
from sklearn.metrics import roc_auc_score
# prediction_path = "predictions"

""" Save the results in this folder """
# create_dir("results_sunil_test")
# create_dir(prediction_path)

""" Load the model """
# with CustomObjectScope({'iou': iou, 'dice_coef': dice_coef}):
model_path = os.path.join(f"{model_dir}", "model.h5")

model = load_model(model_path, compile=False)

""" Load the dataset """
dataset_path = os.path.join(f"{data_path}/{split_name}", "test")
test_x, test_y = load_data(dataset_path)

""" Make the prediction and calculate the metrics values """
SCORE = []
for x, y in tqdm(zip(test_x, test_y), total=len(test_x)):
    """ Extracting name """
    name = x.split("/")[-1].split(".")[0]

    """ Read the image and mask """
    ori_x, x = read_image(x)
    ori_y, y = read_mask(y)
    y = y.astype(np.int)

    """ Prediction """
    y_pred = model.predict(np.expand_dims(x, axis=0))[0]
    np.save(f"{prediction_np_path}/{name}.npy", y_pred)
    y_prob = deepcopy(y_pred).flatten()
    y_pred = y_pred > 0.5
    y_pred = y_pred.astype(np.int32)
    y_pred = np.squeeze(y_pred, axis=-1)

    """ Saving the images """
    save_image_path = f"{results_path}/{name}.png"
#    print(ori_x.shape, ori_y.shape, y_pred.shape)
    save_results(ori_x, ori_y, y_pred, save_image_path)
    cv2.imwrite(f"{prediction_path}/{name}.png", y_pred * 255)
    

    """ Flatten the array """
    y = y.flatten()
    y_pred = y_pred.flatten()

    """ Calculate the metrics """
    acc_value = accuracy_score(y, y_pred)
    f1_value = f1_score(y, y_pred, labels=[0, 1], average="binary")
    jac_value = jaccard_score(y, y_pred, labels=[0, 1], average="binary")
    recall_value = recall_score(y, y_pred, labels=[0, 1], average="binary")
    precision_value = precision_score(y, y_pred, labels=[0, 1], average="binary")
    if np.sum(y) == 0 or np.sum(y) == H * W:
        auc_value = 1
    else:
        auc_value = roc_auc_score(y, y_prob)

    SCORE.append([name, acc_value, f1_value, jac_value, recall_value, precision_value, auc_value])

score = [s[1:] for s in SCORE]
score = np.mean(score, axis=0)
print(f"Accuracy: {score[0]:0.5f}")
print(f"F1: {score[1]:0.5f}")
print(f"Jaccard: {score[2]:0.5f}")
print(f"Recall: {score[3]:0.5f}")
print(f"Precision: {score[4]:0.5f}")
print(f"AUC: {score[5]:0.5f}")

""" Saving """
df = pd.DataFrame(SCORE, columns=["Image", "Acc", "F1", "Jaccard", "Recall", "Precision", "AUC"])
df.to_csv(f"{results_path}/score.csv")

 14%|█▍        | 91/630 [00:45<04:18,  2.08it/s]/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1580: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Jaccard is ill-defined and being set to 0.0 due to no true or predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_clas

Accuracy: 0.97426
F1: 0.75084
Jaccard: 0.61888
Recall: 0.72564
Precision: 0.79800
AUC: 0.98208


In [ ]:
model_dir

# Reconstruct original image

In [ ]:
# import glob
# split = "test"
# image_patches_path = f"{data_path}/{split_name}/{split}/image/"
# #create_dir(image_patches_path)
# #gt_patches_path = f"{data_path}/{split_name}/{split}/mask/"


# reconstructed_original_path = f"{data_path}/{split_name}/reconstructed/{split}/image"
# create_dir(reconstructed_original_path)
# image_sizes = {"HRF": (2336, 3504), "drive": (584, 565), "chase_db": (960, 999)}
# original_filenames = get_filenames_sorted(image_patches_path)[0]
# reconstruct_patches(original_filenames,
#                     image_sizes["HRF"],
#                     reconstructed_original_path,
#                     stride=512,
#                     )


In [ ]:
# gt_patches_path = f"{data_path}/{split_name}/{split}/mask/"
# reconstructed_gt_path = f"{data_path}/{split_name}/reconstructed/{split}/mask"

# create_dir(reconstructed_gt_path)
# gt_filenames = get_filenames_sorted(gt_patches_path)[0]

# reconstruct_patches(gt_filenames,
#                     image_sizes["HRF"],
#                     reconstructed_gt_path,
#                     stride=512,
#                     )


In [ ]:
prediction_np_path

In [ ]:
from glob import glob

In [ ]:
# Create patches dictionary
predicted_patches_dict = dict()
for file in glob(f"{prediction_np_path}/*.npy"):
    print("file", file)
    patch = np.load(file)
    fname_without_ext = file.rsplit("/", 1)[1].rsplit(".", 1)[0]
    fname_without_ext = fname_without_ext[1:]
    patch_number = int(fname_without_ext.rsplit("_", 1)[1])
    image_file = fname_without_ext.rsplit("_", 1)[0]
    print(image_file, patch_number )

    if image_file in predicted_patches_dict.keys():
        predicted_patches_dict[image_file][patch_number] = np.load(file)
    else:
        patches = [None] * 140
        patches[patch_number] = np.load(file)
        predicted_patches_dict[image_file] = patches

        




In [ ]:
# def reconstruct_patches(patches_dict, metadata_train_df, out_path, mfactor=1 ):
#     rows_per_image = metadata_train_df.groupby(["image_filename"])["row"].apply(list)
#     cols_per_image = metadata_train_df.groupby(["image_filename"])["col"].apply(list)

#     for r_row, c_row in zip(rows_per_image.iteritems(), cols_per_image.iteritems()):
#         image_filename = r_row[0]
#         print(image_filename)
#         if r_row[0] != c_row[0] :
#             raise Exception("Filename in Row and column index does not match")
#         patch_height = metadata_train_df[metadata_train_df["image_filename"] == image_filename]["patch_height"].max()
#         patch_width = metadata_train_df[metadata_train_df["image_filename"] == image_filename]["patch_width"].max()
#         height = max(r_row[1]) + patch_height
#         width = max(c_row[1]) + patch_width
#         im = np.zeros([height, width, 3])
#         count_im = np.zeros([height, width])
#         filename = image_filename.rsplit("/", 1)[1]
#         name = filename.rsplit(".", 1)[0]
#         patches = patches_dict[name]
#         patch_count = 0
#         print(r_row[1])
#         print(c_row[1])
#         for row , col in zip(r_row[1], c_row[1]):
#             count_im[row:row + patch_height, col:col + patch_width] += np.ones((patch_height, patch_width))
#             im[row:row + patch_height, col:col + patch_width] += patches[patch_count]
#             patch_count += 1

#         for channel in range(3):
#             im[:, :, channel] = np.true_divide(im[:,:, channel], count_im)
        
#         print("Num patches added ", patch_count)
#         cv2.imwrite(f"{out_path}/{name}.png", im * mfactor)
        


In [ ]:
# !rm -rf /content/datasets/hrf/split_2/reconstructed/test/predictions_reconstructed

In [ ]:
reconstructed_predictions_path = f"{data_path}/{split_name}/reconstructed/{split}/predictions_reconstructed"
create_dir(reconstructed_predictions_path)
reconstruct_patches(predicted_patches_dict,
                    metadata_test_df,
                    reconstructed_predictions_path,
                    mfactor=255
                    )


In [ ]:
!pwd

/content/datasets


In [ ]:
def read_mask(path):
    print(path)
    x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)  ## (512, 512)
    # x = cv2.resize(x, (W, H))
    ori_x = x
    x = x/255.0
#    x = x.astype(np.int32)
    return ori_x, x

In [ ]:
def save_results(ori_x, ori_y, y_pred, save_image_path):
    line = np.ones((2560, 10, 3)) * 255

    ori_y = np.expand_dims(ori_y, axis=-1)
    ori_y = np.concatenate([ori_y, ori_y, ori_y], axis=-1)

    y_pred = np.expand_dims(y_pred, axis=-1)
    y_pred = np.concatenate([y_pred, y_pred, y_pred], axis=-1) * 255

    cat_images = np.concatenate([ori_x, line, ori_y, line, y_pred], axis=1)
    print(f"Saving image of size {cat_images.shape} to {save_image_path}")
    cv2.imwrite(save_image_path, cat_images)

In [ ]:
results_path_reconstructed = f"{root_path}/results_reconstructed/"
create_dir(results_path_reconstructed)

In [ ]:
from glob import glob

In [ ]:
from copy import deepcopy
from sklearn.metrics import roc_auc_score
# prediction_path = "predictions"

""" Load the model """
# with CustomObjectScope({'iou': iou, 'dice_coef': dice_coef}):
model_path = os.path.join(f"{model_dir}", "model.h5")
model = load_model(model_path, compile=False)

""" Load the dataset """
# dataset_path = os.path.join(f"{data_path}/{split_name}", "test")

test_x, test_y = load_data(f"{data_path}/{split_name}/reconstructed/test/")

""" Make the prediction and calculate the metrics values """
SCORE = []
for x, y in tqdm(zip(test_x, test_y), total=len(test_x)):
    """ Extracting name """
    name = x.split("/")[-1].split(".")[0]

    """ Read the image and mask """
    ori_x, x = read_image(x)
    ori_y, y = read_mask(y)
    y = y.astype(np.int)
    print(name)

#     """ Prediction """
    #y_pred = model.predict(np.expand_dims(x, axis=0))[0]

    orig_y_pred, y_pred = read_mask(f"{data_path}/{split_name}/reconstructed/test/predictions_reconstructed/{name}.png")
    print(y_pred.shape, np.max(y_pred), np.min(y_pred))
    print(y_pred.shape, np.max(orig_y_pred), np.min(orig_y_pred))
    y_prob = deepcopy(y_pred).flatten()
    y_pred = y_pred > 0.5
    y_pred = y_pred.astype(np.int32)

    """ Saving the images """
    save_image_path = f"{results_path_reconstructed}/{name}.png"
    print(ori_x.shape, ori_y.shape,y_pred.shape)
    save_results(ori_x, ori_y, y_pred, save_image_path)

    
#    cv2.imwrite(f"{prediction_path}/{name}.png", y_pred * 255)
    

    """ Flatten the array """
    y = y.flatten()
    y_pred = y_pred.flatten()

    """ Calculate the metrics """
    acc_value = accuracy_score(y, y_pred)
    f1_value = f1_score(y, y_pred, labels=[0, 1], average="binary")
    jac_value = jaccard_score(y, y_pred, labels=[0, 1], average="binary")
    recall_value = recall_score(y, y_pred, labels=[0, 1], average="binary")
    precision_value = precision_score(y, y_pred, labels=[0, 1], average="binary")
    if np.sum(y) == 0 or np.sum(y) == H * W:
        auc_value = 1
    else:
        auc_value = roc_auc_score(y, y_prob)

    SCORE.append([name, acc_value, f1_value, jac_value, recall_value, precision_value, auc_value])

score = [s[1:] for s in SCORE]
score = np.mean(score, axis=0)
print(f"Accuracy: {score[0]:0.5f}")
print(f"F1: {score[1]:0.5f}")
print(f"Jaccard: {score[2]:0.5f}")
print(f"Recall: {score[3]:0.5f}")
print(f"Precision: {score[4]:0.5f}")
print(f"AUC: {score[5]:0.5f}")

""" Saving """
df = pd.DataFrame(SCORE, columns=["Image", "Acc", "F1", "Jaccard", "Recall", "Precision", "AUC"])
df.to_csv(f"{results_path_reconstructed}/score_full.csv")

  0%|          | 0/18 [00:00<?, ?it/s]

/content/datasets/hrf_512_512_bce_hvr//split_2/reconstructed/test/mask/01_h.png
01_h
/content/datasets/hrf_512_512_bce_hvr//split_2/reconstructed/test/predictions_reconstructed/01_h.png
(2560, 3584) 1.0 0.0
(2560, 3584) 255 0
(2560, 3584, 3) (2560, 3584) (2560, 3584)
Saving image of size (2560, 10772, 3) to /content/datasets/hrf_512_512_bce_hvr//results_reconstructed//01_h.png


  6%|▌         | 1/18 [00:16<04:33, 16.06s/it]

/content/datasets/hrf_512_512_bce_hvr//split_2/reconstructed/test/mask/02_g.png
02_g
/content/datasets/hrf_512_512_bce_hvr//split_2/reconstructed/test/predictions_reconstructed/02_g.png
(2560, 3584) 1.0 0.0
(2560, 3584) 255 0
(2560, 3584, 3) (2560, 3584) (2560, 3584)
Saving image of size (2560, 10772, 3) to /content/datasets/hrf_512_512_bce_hvr//results_reconstructed//02_g.png


 11%|█         | 2/18 [00:32<04:18, 16.17s/it]

...

/content/datasets/hrf_512_512_bce_hvr//split_2/reconstructed/test/mask/15_g.png
15_g
/content/datasets/hrf_512_512_bce_hvr//split_2/reconstructed/test/predictions_reconstructed/15_g.png
(2560, 3584) 1.0 0.0
(2560, 3584) 255 0
(2560, 3584, 3) (2560, 3584) (2560, 3584)
Saving image of size (2560, 10772, 3) to /content/datasets/hrf_512_512_bce_hvr//results_reconstructed//15_g.png


 94%|█████████▍| 17/18 [04:32<00:16, 16.09s/it]

/content/datasets/hrf_512_512_bce_hvr//split_2/reconstructed/test/mask/15_h.png
15_h
/content/datasets/hrf_512_512_bce_hvr//split_2/reconstructed/test/predictions_reconstructed/15_h.png
(2560, 3584) 1.0 0.0
(2560, 3584) 255 0
(2560, 3584, 3) (2560, 3584) (2560, 3584)
Saving image of size (2560, 10772, 3) to /content/datasets/hrf_512_512_bce_hvr//results_reconstructed//15_h.png


100%|██████████| 18/18 [04:48<00:00, 16.05s/it]

Accuracy: 0.97426
F1: 0.80231
Jaccard: 0.67067
Recall: 0.77847
Precision: 0.83466
AUC: 0.97842


In [ ]:
from copy import deepcopy
from sklearn.metrics import roc_auc_score
# prediction_path = "predictions"

""" Load the model """
# with CustomObjectScope({'iou': iou, 'dice_coef': dice_coef}):
model_path = os.path.join(f"{model_dir}", "model.h5")
model = load_model(model_path, compile=False)

""" Load the dataset """
# dataset_path = os.path.join(f"{data_path}/{split_name}", "test")

test_x, test_y = load_data(f"{data_path}/{split_name}/reconstructed/test/")


""" Make the prediction and calculate the metrics values """
SCORE = []
for x, y in tqdm(zip(test_x, test_y), total=len(test_x)):
    """ Extracting name """
    name = x.split("/")[-1].split(".")[0]

    """ Read the image and mask """
    ori_x, x = read_image(x)
    ori_y, y = read_mask(y)
    y = y.astype(np.int)
    print(name)

#     """ Prediction """
    #y_pred = model.predict(np.expand_dims(x, axis=0))[0]

    orig_y_pred, y_pred = read_mask(f"{data_path}/{split_name}/reconstructed/test/predictions_reconstructed/{name}.png")
    print(y_pred.shape, np.max(y_pred), np.min(y_pred))
    print(y_pred.shape, np.max(orig_y_pred), np.min(orig_y_pred))
    y_prob = deepcopy(y_pred).flatten()
    y_pred = y_pred > 0.6
    y_pred = y_pred.astype(np.int32)

    """ Saving the images """
    save_image_path = f"{results_path_reconstructed}/{name}.png"
    print(ori_x.shape, ori_y.shape,y_pred.shape)
    save_results(ori_x, ori_y, y_pred, save_image_path)

    
#    cv2.imwrite(f"{prediction_path}/{name}.png", y_pred * 255)
    

    """ Flatten the array """
    y = y.flatten()
    y_pred = y_pred.flatten()

    """ Calculate the metrics """
    acc_value = accuracy_score(y, y_pred)
    f1_value = f1_score(y, y_pred, labels=[0, 1], average="binary")
    jac_value = jaccard_score(y, y_pred, labels=[0, 1], average="binary")
    recall_value = recall_score(y, y_pred, labels=[0, 1], average="binary")
    precision_value = precision_score(y, y_pred, labels=[0, 1], average="binary")
    if np.sum(y) == 0 or np.sum(y) == H * W:
        auc_value = 1
    else:
        auc_value = roc_auc_score(y, y_prob)

    SCORE.append([name, acc_value, f1_value, jac_value, recall_value, precision_value, auc_value])

score = [s[1:] for s in SCORE]
score = np.mean(score, axis=0)
print(f"Accuracy: {score[0]:0.5f}")
print(f"F1: {score[1]:0.5f}")
print(f"Jaccard: {score[2]:0.5f}")
print(f"Recall: {score[3]:0.5f}")
print(f"Precision: {score[4]:0.5f}")
print(f"AUC: {score[5]:0.5f}")

""" Saving """
df = pd.DataFrame(SCORE, columns=["Image", "Acc", "F1", "Jaccard", "Recall", "Precision", "AUC"])
df.to_csv(f"{results_path_reconstructed}/score_full.csv")

  0%|          | 0/18 [00:00<?, ?it/s]

/content/datasets/hrf_512_512_bce_hvr//split_2/reconstructed/test/mask/01_h.png
01_h
/content/datasets/hrf_512_512_bce_hvr//split_2/reconstructed/test/predictions_reconstructed/01_h.png
(2560, 3584) 1.0 0.0
(2560, 3584) 255 0
(2560, 3584, 3) (2560, 3584) (2560, 3584)
Saving image of size (2560, 10772, 3) to /content/datasets/hrf_512_512_bce_hvr//results_reconstructed//01_h.png


  6%|▌         | 1/18 [00:16<04:41, 16.53s/it]

/content/datasets/hrf_512_512_bce_hvr//split_2/reconstructed/test/mask/02_g.png
02_g
/content/datasets/hrf_512_512_bce_hvr//split_2/reconstructed/test/predictions_reconstructed/02_g.png
(2560, 3584) 1.0 0.0
(2560, 3584) 255 0
(2560, 3584, 3) (2560, 3584) (2560, 3584)
Saving image of size (2560, 10772, 3) to /content/datasets/hrf_512_512_bce_hvr//results_reconstructed//02_g.png


 11%|█         | 2/18 [00:32<04:23, 16.48s/it]

...

/content/datasets/hrf_512_512_bce_hvr//split_2/reconstructed/test/mask/15_g.png
15_g
/content/datasets/hrf_512_512_bce_hvr//split_2/reconstructed/test/predictions_reconstructed/15_g.png
(2560, 3584) 1.0 0.0
(2560, 3584) 255 0
(2560, 3584, 3) (2560, 3584) (2560, 3584)
Saving image of size (2560, 10772, 3) to /content/datasets/hrf_512_512_bce_hvr//results_reconstructed//15_g.png


 94%|█████████▍| 17/18 [04:38<00:16, 16.28s/it]

/content/datasets/hrf_512_512_bce_hvr//split_2/reconstructed/test/mask/15_h.png
15_h
/content/datasets/hrf_512_512_bce_hvr//split_2/reconstructed/test/predictions_reconstructed/15_h.png
(2560, 3584) 1.0 0.0
(2560, 3584) 255 0
(2560, 3584, 3) (2560, 3584) (2560, 3584)
Saving image of size (2560, 10772, 3) to /content/datasets/hrf_512_512_bce_hvr//results_reconstructed//15_h.png


100%|██████████| 18/18 [04:54<00:00, 16.36s/it]

Accuracy: 0.97402
F1: 0.79314
Jaccard: 0.65784
Recall: 0.74127
Precision: 0.86090
AUC: 0.97842


In [ ]:
from copy import deepcopy
from sklearn.metrics import roc_auc_score
# prediction_path = "predictions"

""" Load the model """
# with CustomObjectScope({'iou': iou, 'dice_coef': dice_coef}):
model_path = os.path.join(f"{model_dir}", "model.h5")
model = load_model(model_path, compile=False)

""" Load the dataset """
# dataset_path = os.path.join(f"{data_path}/{split_name}", "test")

test_x, test_y = load_data(f"{data_path}/{split_name}/reconstructed/test/")


""" Make the prediction and calculate the metrics values """
SCORE = []
for x, y in tqdm(zip(test_x, test_y), total=len(test_x)):
    """ Extracting name """
    name = x.split("/")[-1].split(".")[0]

    """ Read the image and mask """
    ori_x, x = read_image(x)
    ori_y, y = read_mask(y)
    y = y.astype(np.int)
    print(name)

#     """ Prediction """
    #y_pred = model.predict(np.expand_dims(x, axis=0))[0]

    orig_y_pred, y_pred = read_mask(f"{data_path}/{split_name}/reconstructed/test/predictions_reconstructed/{name}.png")
    print(y_pred.shape, np.max(y_pred), np.min(y_pred))
    print(y_pred.shape, np.max(orig_y_pred), np.min(orig_y_pred))
    y_prob = deepcopy(y_pred).flatten()
    y_pred = y_pred > 0.4
    y_pred = y_pred.astype(np.int32)

    """ Saving the images """
    save_image_path = f"{results_path_reconstructed}/{name}.png"
    print(ori_x.shape, ori_y.shape,y_pred.shape)
    save_results(ori_x, ori_y, y_pred, save_image_path)

    
    # cv2.imwrite(f"{prediction_path}/{name}.png", y_pred * 255)
    

    """ Flatten the array """
    y = y.flatten()
    y_pred = y_pred.flatten()

    """ Calculate the metrics """
    acc_value = accuracy_score(y, y_pred)
    f1_value = f1_score(y, y_pred, labels=[0, 1], average="binary")
    jac_value = jaccard_score(y, y_pred, labels=[0, 1], average="binary")
    recall_value = recall_score(y, y_pred, labels=[0, 1], average="binary")
    precision_value = precision_score(y, y_pred, labels=[0, 1], average="binary")
    if np.sum(y) == 0 or np.sum(y) == H * W:
        auc_value = 1
    else:
        auc_value = roc_auc_score(y, y_prob)

    SCORE.append([name, acc_value, f1_value, jac_value, recall_value, precision_value, auc_value])

score = [s[1:] for s in SCORE]
score = np.mean(score, axis=0)
print(f"Accuracy: {score[0]:0.5f}")
print(f"F1: {score[1]:0.5f}")
print(f"Jaccard: {score[2]:0.5f}")
print(f"Recall: {score[3]:0.5f}")
print(f"Precision: {score[4]:0.5f}")
print(f"AUC: {score[5]:0.5f}")

""" Saving """
df = pd.DataFrame(SCORE, columns=["Image", "Acc", "F1", "Jaccard", "Recall", "Precision", "AUC"])
df.to_csv(f"{results_path_reconstructed}/score_full.csv")

  0%|          | 0/18 [00:00<?, ?it/s]

/content/datasets/hrf_512_512_bce_hvr//split_2/reconstructed/test/mask/01_h.png
01_h
/content/datasets/hrf_512_512_bce_hvr//split_2/reconstructed/test/predictions_reconstructed/01_h.png
(2560, 3584) 1.0 0.0
(2560, 3584) 255 0
(2560, 3584, 3) (2560, 3584) (2560, 3584)
Saving image of size (2560, 10772, 3) to /content/datasets/hrf_512_512_bce_hvr//results_reconstructed//01_h.png


  6%|▌         | 1/18 [00:16<04:43, 16.67s/it]

/content/datasets/hrf_512_512_bce_hvr//split_2/reconstructed/test/mask/02_g.png
02_g
/content/datasets/hrf_512_512_bce_hvr//split_2/reconstructed/test/predictions_reconstructed/02_g.png
(2560, 3584) 1.0 0.0
(2560, 3584) 255 0
(2560, 3584, 3) (2560, 3584) (2560, 3584)
Saving image of size (2560, 10772, 3) to /content/datasets/hrf_512_512_bce_hvr//results_reconstructed//02_g.png


 11%|█         | 2/18 [00:33<04:23, 16.50s/it]

...

/content/datasets/hrf_512_512_bce_hvr//split_2/reconstructed/test/mask/15_g.png
15_g
/content/datasets/hrf_512_512_bce_hvr//split_2/reconstructed/test/predictions_reconstructed/15_g.png
(2560, 3584) 1.0 0.0
(2560, 3584) 255 0
(2560, 3584, 3) (2560, 3584) (2560, 3584)
Saving image of size (2560, 10772, 3) to /content/datasets/hrf_512_512_bce_hvr//results_reconstructed//15_g.png


 94%|█████████▍| 17/18 [04:37<00:16, 16.25s/it]

/content/datasets/hrf_512_512_bce_hvr//split_2/reconstructed/test/mask/15_h.png
15_h
/content/datasets/hrf_512_512_bce_hvr//split_2/reconstructed/test/predictions_reconstructed/15_h.png
(2560, 3584) 1.0 0.0
(2560, 3584) 255 0
(2560, 3584, 3) (2560, 3584) (2560, 3584)
Saving image of size (2560, 10772, 3) to /content/datasets/hrf_512_512_bce_hvr//results_reconstructed//15_h.png


100%|██████████| 18/18 [04:53<00:00, 16.32s/it]

Accuracy: 0.97387
F1: 0.80601
Jaccard: 0.67613
Recall: 0.81001
Precision: 0.80826
AUC: 0.97842


In [ ]:
history_df = pd.read_csv(csv_path)

In [ ]:
history_df.shape

(50, 14)

In [ ]:
history_df.head(20)

In [ ]:
results_path

In [ ]:
!ls 

hrf_512_512_bce_hvr  logs


In [ ]:
!mv hrf_512_512_bce_hvr hrf_512_512_bce_hvr_feb_3_1

In [ ]:
!zip -r hrf_512_512_bce_hvr_feb_3_1.zip  hrf_512_512_bce_hvr_feb_3_1/

Streaming output truncated to the last 5000 lines.
  adding: hrf_512_512_bce_hvr_feb_3_1/split_2/train/mask/_01_dr_1_8.png (deflated 31%)
  adding: hrf_512_512_bce_hvr_feb_3_1/split_2/train/mask/_07_h_3_25.png (deflated 26%)
  adding: hrf_512_512_bce_hvr_feb_3_1/split_2/train/mask/_11_g_3_15.png (deflated 30%)
  adding: hrf_512_512_bce_hvr_feb_3_1/split_2/train/mask/_10_g_3_8.png (deflated 30%)
  adding: hrf_512_512_bce_hvr_feb_3_1/split_2/train/mask/_14_dr_3_28.png (deflated 56%)
  adding: hrf_512_512_bce_hvr_feb_3_1/split_2/train/mask/_10_dr_1_17.png (deflated 32%)
  adding: hrf_512_512_bce_hvr_feb_3_1/split_2/train/mask/_02_h_0_27.png (deflated 31%)
  adding: hrf_512_512_bce_hvr_feb_3_1/split_2/train/mask/_10_g_1_1.png (deflated 35%)
  adding: hrf_512_512_bce_hvr_feb_3_1/split_2/train/mask/_14_h_0_13.png (deflated 33%)
  adding: hrf_512_512_bce_hvr_feb_3_1/split_2/train/mask/_09_g_0_14.png (deflated 34%)
  adding: hrf_512_512_bce_hvr_feb_3_1/split_2/train/mask/_10_dr_1_9.png (deflat

In [ ]:
!ls -ltrh


total 2.8G
drwxr-xr-x  4 root root 4.0K Feb  3 02:43 logs
drwxr-xr-x 11 root root 4.0K Feb  3 09:07 hrf_512_512_bce_hvr_feb_3_1
-rw-r--r--  1 root root 2.8G Feb  3 10:13 hrf_512_512_bce_hvr_feb_3_1.zip


In [ ]:
!ls /content/gdrive/MyDrive/Blood_Vessel_Results/hrf_patches/

02_g_test.png
hrf_512_256_bce_hvr_xval_10_42.zip
hrf_512_512
hrf_512_512_bce
hrf_512_512_bce_hvr
hrf_512_512_bce_hvr_0_3
hrf_512_512_bce_hvr_0_33_42.zip
hrf_512_512_bce_hvr_0_33_43
hrf_512_512_bce_hvr_0_33_43.zip
hrf_512_512_bce_hvr_0_3_44
hrf_512_512_bce_hvr_0_3_44.zip
hrf_512_512_bce_hvr_xval_2_42.zip
hrf_512_512_bce_hvr_xval_5_42
hrf_512_512_bce_hvr_xval_5_42_10epochs
hrf_512_512_bce_hvr_xval_5_42.csv
hrf_512_512_bce_hvr.zip
hrf_512_512_bce.zip
hrf_512_512_dice
hrf_512_512_dice_bce
hrf_512_512_dice_bce_hvr
hrf_512_512_dice_bce_hvr_2.zip
hrf_512_512_dice_bce_hvr_jan_31_2.zip
hrf_512_512_dice_bce_hvr_jan_31.zip
hrf_512_512_dice_bce_hvr_xval_5_42.zip
hrf_512_512_dice_bce_hvr_xval_9_42_10epochs.zip
hrf_512_512_dice_bce_hvr.zip
hrf_512_512_DiceBCELoss
hrf_512_512_dice_loss_2
hrf_512_512_dice_loss_2.zip
hrf_512_512_dice_loss_hvr_xval_5_42.zip
hrf_512_512_dice.zip
hrf_512_512.zip
hrf_patches_512.zip
hrf.zip
non_overlapping_512
split_2.zip


In [ ]:
!cp hrf_512_512_bce_hvr_feb_3_1.zip /content/gdrive/MyDrive/Blood_Vessel_Results/hrf_patches/

In [ ]:
# %cd /content/gdrive/MyDrive/Blood_Vessel_Results/hrf_patches/

In [ ]:
!pwd

/content/datasets


In [ ]:
!ls

hrf_512_512_bce_hvr_feb_3_1  hrf_512_512_bce_hvr_feb_3_1.zip  logs


In [ ]:
drive.flush_and_unmount()